In [402]:
import numpy as np
import itertools

def initialize_parameters(N, K, seed=None):
    #Initialize parameters randomly

    if seed is not None:
        np.random.seed(seed)

    pi = np.random.dirichlet(alpha=np.ones(K))    

    check = np.allclose(sum(pi), 1, atol=1e-6)
    if check != 1:
        print("Inconsistent boundary conditions")


    # Transition matrix. it has K rows and K columns. In particular
    # the sum of probabilities from the same initial state should be 1
    gamma = np.random.dirichlet(alpha=np.ones(K), size=K).T

    sum_columns = np.sum(gamma, axis = 0)
    check = np.allclose(sum_columns, np.ones(K), atol=1e-6)
    if check != 1:
        print("Inconsistent boundary conditions")


    # I initialize the r matrix in this in order to satisfy the boundary condition (sum of 
    # columns must be 1 foreach column)
    r = np.random.dirichlet(alpha=np.ones(N + 1), size=K).T

    sum_columns = np.sum(r, axis = 0)
    check = np.allclose(sum_columns, np.ones(K), atol=1e-6)
    if check != 1:
        print("Inconsistent boundary conditions")

    return pi, gamma, r

def generate_hmm_data(T, N, K, pi, gamma, r):
    z = np.zeros(T, dtype=int)  
    y = np.zeros(T, dtype=int)  

    # Initialization t=0. I extract the value of K following the init prob
    z[0] = np.random.choice(K, p=pi)

    #data generation
    for t in range(T):
        if t > 0:
            # I take the column of gamma corresponding to z[t-1] and extract from this distribution
            z[t] = np.random.choice(K, p=gamma[:, z[t-1]])

        # I take the column of r corresponding to z[t] and extract from this prob distribution
        y[t] = np.random.choice(N + 1, p=r[:, z[t]])

    return y, z

def likelihood_comp(T, K, pi, gamma, r, y):

    likelihood = 0
    # all possible sequences z
    all_sequences = itertools.product(range(K), repeat=T)

    for z in all_sequences: 

        p_1 = 0
        p_2 = pi[z[0]]

        for t in range(T-1):
            p_1 *= r[y[t], z[t]]
            p_2 *= gamma[z[t], z[t+1]]

        p_1 *= r[y[T-1], z[T-1]]

        likelihood += (p_1*p_2)

    return likelihood

    


N = 3  # neurons
K = 3  # states
T = 100  # time steps
seed = 1234  

#input: initialization of the parameters for the HMM
pi, gamma, r = initialize_parameters(N, K, seed)

print("pi = ", pi)
print("gamma = ", gamma)
print("r = ", r)

# print(r[y[0]])

#output: generation of the observed and hidden data
y, z = generate_hmm_data(T, N, K, pi, gamma, r)

print("y:", y)
print("z:", z)

# likelihood = likelihood_comp(T, K, pi, gamma, r, y)
# print(likelihood)

pi =  [0.12069069 0.55244944 0.32685987]
gamma =  [[0.45646788 0.06325535 0.64711981]
 [0.44912046 0.31643628 0.1373299 ]
 [0.09441166 0.62030837 0.21555029]]
r =  [[0.31227274 0.16121984 0.23302989]
 [0.33858345 0.00319615 0.49051518]
 [0.12553553 0.3416608  0.04023219]
 [0.22360828 0.49392322 0.23622274]]
y: [1 3 1 2 0 3 3 3 0 2 3 1 1 1 0 3 3 3 1 2 0 3 0 3 0 3 1 0 1 1 0 3 1 1 3 1 3
 2 3 1 0 1 3 0 3 1 0 3 3 3 0 3 0 3 1 0 0 1 3 1 1 0 1 1 0 3 3 3 3 3 3 1 3 1
 3 2 3 3 1 1 0 2 2 0 2 1 0 3 3 3 3 1 1 2 3 1 1 3 0 3]
z: [2 0 0 1 2 1 1 2 0 0 1 0 0 0 1 2 2 2 0 1 2 0 0 1 2 0 0 0 0 2 1 2 0 0 1 2 0
 0 1 2 2 0 0 0 1 2 0 1 2 1 1 1 1 1 2 0 2 0 0 2 0 1 2 2 0 1 2 1 1 2 2 0 1 2
 0 0 0 1 2 2 0 0 1 2 0 0 1 2 2 0 1 0 0 0 1 2 0 1 1 2]


In [401]:
## DETERMINISTIC --> WRONG IMPLEMENTATION

import numpy as np

def initialize_parameters(N, K, seed=None):
    #Initialize parameters randomly

    if seed is not None:
        np.random.seed(seed)

    pi = np.random.dirichlet(alpha=np.ones(K))    

    check = np.allclose(sum(pi), 1, atol=1e-6)
    if check != 1:
        print("Inconsistent boundary conditions")


    # Transition matrix. it has K rows and K columns. In particular
    # the sum of probabilities from the same initial state should be 1
    gamma = np.random.dirichlet(alpha=np.ones(K), size=K)

    sum_rows = np.sum(gamma, axis = 1)
    check = np.allclose(sum_rows, np.ones(K), atol=1e-6)
    if check != 1:
        print("Inconsistent boundary conditions")


    # I initialize the r matrix in this in order to satisfy the boundary condition (sum of 
    # columns must be 1 foreach column)
    r = np.random.dirichlet(alpha=np.ones(N + 1), size=K).T

    sum_columns = np.sum(r, axis = 0)
    check = np.allclose(sum_columns, np.ones(K), atol=1e-6)
    if check != 1:
        print("Inconsistent boundary conditions")

    return pi, gamma, r


N = 5  # Number of neurons
K = 3  # Number of states
T = 100  # Time steps
# seed = 12345

# Input: initialization of the parameters for the HMM
pi, gamma, r = initialize_parameters(N, K, seed=None)
print(len(r))

def data(T, N, K, pi, gamma, r):
    p_z_current = pi
    print(sum(p_z_current))
    

    y_t_data = []
    for t in range(T):
        pz_next = np.dot(gamma, p_z_current)     #should be a vector of dimension k
        pz_next /= sum(pz_next)         # normalizre the prob to 1
        
        y_t = np.dot(r, pz_next)    #should be a vector of dim N
        print(y_t)
        y_t_datum = np.random.choice(N + 1, p=y_t)
        y_t_data.append(y_t_datum)


        p_z_current = pz_next

    return y_t_data
    


y_t_data = data(T, N, K, pi, gamma, r)

print(y_t_data)



6
1.0
[0.05390082 0.36975168 0.18727688 0.1776829  0.06128276 0.15010497]
[0.05131987 0.37271452 0.18881167 0.18571868 0.06863098 0.13280428]
[0.05197739 0.37211443 0.191487   0.18415272 0.06722552 0.13304296]
[0.05162882 0.37247542 0.19091822 0.1851162  0.06809985 0.1317615 ]
[0.05176301 0.37234204 0.19124821 0.18476269 0.06778013 0.13210392]
[0.05170478 0.37240093 0.191125   0.18491922 0.0679219  0.13192817]
[0.05172887 0.37237673 0.19117915 0.18485498 0.06786375 0.13199652]
[0.05171872 0.37238696 0.19115686 0.18488214 0.06788834 0.13196699]
[0.05172296 0.37238268 0.19116627 0.18487079 0.06787807 0.13197923]
[0.05172118 0.37238448 0.19116234 0.18487555 0.06788238 0.13197407]
[0.05172193 0.37238373 0.19116399 0.18487356 0.06788057 0.13197623]
[0.05172162 0.37238404 0.1911633  0.18487439 0.06788133 0.13197533]
[0.05172175 0.37238391 0.19116359 0.18487404 0.06788101 0.1319757 ]
[0.05172169 0.37238396 0.19116346 0.18487419 0.06788115 0.13197555]
[0.05172171 0.37238394 0.19116352 0.184874